# Politician Activity on Wikipedia by Political Affiliation

The parameters in the cell below can be adjusted to explore other political affiliations and time frames.

### How to explore other political affiliation?
The ***affiliation*** parameter can be use to aggregate politicians by their political affiliations. The column `affiliation` in this [this other notebook](../politicians.ipynb?autorun=true) show the politicians that belong each political affiliation.

***Alternatively***, you can direcly use the [politicians API](http://mediamonitoring.gesis.org/api/politicians/swagger/), or access it with the [SMM Wrapper](https://pypi.org/project/smm-wrapper/).

In [ ]:
# Parameters: 
affiliation = 'CSU'
aggregation = 'week' # aggregation can be by day ('day'), week ('week') or month('month')
from_date='2017-09-01'
to_date='2018-12-31'

## Using API

In [ ]:
import pandas as pd
from smm_wrapper import SMMPoliticians
from wikiwho_wrapper import WikiWho

# Create an instance to the smm wrapper
smm = SMMPoliticians()

#using the api to get the data
df = smm.dv.get_politicians()

# Filter the accounts by party, and valid ones (the ones that contain wp_ids)
party_df = df[(df['affiliation']==affiliation) & (df['wp_ids'].notnull())] 

# query the Social Media Monitoring API
wiki_chobs = pd.concat(smm.dv.wikipedia(wikipedia_page_id=wp_id, from_date=from_date, to_date=to_date, aggregate_by=aggregation) 
               for wp_ids in party_df['wp_ids'] for wp_id in wp_ids) 
wiki_chobs = wiki_chobs.groupby('date').agg({'chobs': 'sum'}).reset_index()

#using wikiwho to extract conflicts and revisions
ww = WikiWho(lng='de')
wiki_data = pd.concat(ww.dv.edit_persistence(page_id=wp_id, start=from_date, end=to_date) for wp_ids in party_df['wp_ids'] for wp_id in wp_ids )
wiki_data['undos'] = wiki_data['dels'] + wiki_data['reins']
wiki_data = wiki_data[['year_month','conflict', 'elegibles', 'revisions', 'undos']]
wiki_data['date'] = pd.to_datetime(wiki_data['year_month'])
wiki_data = wiki_data.groupby('date').agg({'conflict': 'sum','elegibles':'sum', 'revisions':'sum','undos':'sum' }).reset_index()
wiki_data['conflict_score'] = wiki_data['conflict'] / wiki_data['elegibles']

In [ ]:
import requests
appended_views = []
session = requests.Session()
for wp_titles in party_df['wp_titles']:
    for wp_title in wp_titles:
        try:
            
            view_url = 'https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/de.wikipedia.org/all-access/user/{}/daily/{}/{}'.format(wp_title,from_date.replace('-',''),to_date.replace('-',''))
            view = pd.DataFrame(session.get(url=view_url).json()['items'])
            appended_views.append(view)            
          
        except Exception as e:
            #print(view_url)
            pass
            
views = pd.concat(appended_views)
views = views[['timestamp', 'views']]
views['timestamp']=pd.to_datetime(views['timestamp'], format='%Y%m%d%H')
if aggregation  == 'week':    
    views = views.groupby([pd.Grouper(key='timestamp', freq='W-SUN')])['views'].sum().reset_index().sort_values('timestamp')
    views['timestamp'] = views['timestamp'] - pd.Timedelta(days=6)
    
elif aggregation  == 'month':    
    views = views.groupby([pd.Grouper(key='timestamp', freq='MS')])['views'].sum().reset_index().sort_values('timestamp')

## Plotting

In [ ]:
import plotly
from plotly import graph_objs as go
plotly.offline.init_notebook_mode(connected=True)

plotly.offline.iplot({
    "data": [go.Scatter(x=wiki_chobs['date'], y=wiki_chobs['chobs'], name='Changes', line_shape='spline'),
            go.Scatter(x=views['timestamp'], y=views['views'], name='Views', yaxis='y2')], 
    "layout": go.Layout( title='Wikipedia Activity', xaxis={'title':'Date'}, yaxis=dict(
        title='Views and Changes'
    ),
    yaxis2=dict(
        title='Views',
        
        overlaying='y',
        side='right'
    ), showlegend=True)
})
plotly.offline.iplot({
    "data": [go.Scatter(x=wiki_data['date'], y=wiki_data['undos'], name='Undos', line_shape='spline'),
            go.Scatter(x=wiki_data['date'], y=wiki_data['conflict_score'], name='Conflict', line_shape='spline', yaxis='y2')], 
    "layout": go.Layout(title='Wikipedia Disagreement', xaxis={'title':''}, yaxis=dict(
        title='Undos and Conflict'
    ),
    yaxis2=dict(
        title='Conflict',
        
        overlaying='y',
        side='right'
    ), showlegend=True)
})

